## Deliverable 3. Create a Travel Itinerary Map.

In [7]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
from scipy import stats

# Import API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

In [11]:
# 1. Read the WeatherPy_vacation.csv into a DataFrame.
vacation_df = pd.read_csv("Weather_Database/WeatherPy_vacation.csv")
vacation_df.head()

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,1,Kapaa,US,79.00,light rain,22.0752,-159.3190,Sheraton Kauai Resort at Coconut Beach
1,2,Adrar,MR,72.82,overcast clouds,20.5022,-10.0711,NaN
2,3,San Jose Del Palmar,CO,78.80,scattered clouds,4.9742,-76.2283,NaN
3,4,Sambava,MG,78.94,overcast clouds,-14.2667,50.1667,Melrose
4,5,Buta,CD,76.32,overcast clouds,2.7858,24.7300,HOTEL BISO NA BISO


In [15]:
# 2. Prompt the customer for the minimum temperature preference.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
# Prompt the customer for the maximum temperature preference. 
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = vacation_df.loc[(vacation_df["Max Temp"] <= max_temp) & \
                                       (vacation_df["Max Temp"] >= min_temp)]
                                       
preferred_cities_df.head(10)

# 4a. Determine if there are any empty rows.
# pulled from https://stackoverflow.com/questions/27159189/find-empty-or-nan-entry-in-pandas-dataframe
np.where(pd.isnull(preferred_cities_df))

# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna(how='all')

# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat}, {lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    
    except (IndexError):
        print("Hotel not found... skipping.")

# Print the dataframe
hotel_df = pd.DataFrame(hotel_df)
hotel_df.dropna(how='all')

hotel_df.dropna()

# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

# 8a. Create the output File (CSV)
output_data_file = "Weather_Database/WeatherPy_vacation"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 80
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [16]:
# 2. Using the template add the city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 3a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 3b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [17]:
# 4a. Add a marker layer for each city to the map.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)


# 4b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [25]:
# From the map above pick 4 cities and create a vacation itinerary route to travel between the four cities. 
# 5. Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# Hint: The starting and ending city should be the same city.

vacation_start = vacation_df.loc[(vacation_df["Hotel Name"] == "Hotel Tesoro Los Cabos") & (vacation_df["City"] == "Cabo San Lucas")]
vacation_end = vacation_df.loc[(vacation_df["Hotel Name"] == "Hotel Tesoro Los Cabos") & (vacation_df["City"] == "Cabo San Lucas")]
vacation_stop1 = vacation_df.loc[(vacation_df["Hotel Name"] == "TIERRAS") & (vacation_df["City"] == "Nacozari")]
vacation_stop2 = vacation_df.loc[(vacation_df["Hotel Name"] == "Hotel Plaza Providencia") & (vacation_df["City"] == "Ixtapa")]
vacation_stop3 = vacation_df.loc[(vacation_df["Hotel Name"] == "Collection O Hotel Quinta Antigua") & (vacation_df["City"] == "Lazaro Cardenas")]

In [26]:
# 6. Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.
start = (pd.DataFrame(vacation_start).to_numpy()[0][5],pd.DataFrame(vacation_start).to_numpy()[0][6])
end = (pd.DataFrame(vacation_end).to_numpy()[0][5],pd.DataFrame(vacation_end).to_numpy()[0][6])
stop1 = (pd.DataFrame(vacation_stop1).to_numpy()[0][5],pd.DataFrame(vacation_stop1).to_numpy()[0][6])
stop2 = (pd.DataFrame(vacation_stop2).to_numpy()[0][5],pd.DataFrame(vacation_stop2).to_numpy()[0][6])
stop3 = (pd.DataFrame(vacation_stop3).to_numpy()[0][5],pd.DataFrame(vacation_stop3).to_numpy()[0][6])

start

(22.8909, -109.9124)

In [27]:
# 7. Create a direction layer map using the start and end latitude-longitude pairs,
# and stop1, stop2, and stop3 as the waypoints. The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".
fig = gmaps.figure()

start2end_via_stop1_stop2_stop3 = gmaps.directions_layer(start, end, waypoints = [stop1, stop2, stop3], show_markers=True,
    stroke_color='blue', stroke_weight=3.0, stroke_opacity=1.0, travel_mode='DRIVING')
fig.add_layer(start2end_via_stop1_stop2_stop3)
fig

Figure(layout=FigureLayout(height='420px'))

In [29]:
# 8. To create a marker layer map between the four cities.
#  Combine the four city DataFrames into one DataFrame using the concat() function.
itinerary_df = pd.concat([vacation_start, vacation_stop1, vacation_stop2, vacation_stop3],ignore_index=True)
itinerary_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,123,Cabo San Lucas,MX,78.01,clear sky,22.8909,-109.9124,Hotel Tesoro Los Cabos
1,290,Nacozari,MX,75.00,clear sky,25.3366,-108.2574,TIERRAS
2,204,Ixtapa,MX,73.40,overcast clouds,20.7000,-105.2000,Hotel Plaza Providencia
3,476,Lazaro Cardenas,MX,76.62,clear sky,17.9583,-102.2000,Collection O Hotel Quinta Antigua


In [30]:
# 9 Using the template add city name, the country code, the weather description and maximum temperature for the city. 
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = itinerary_df[["Lat", "Lng"]]

In [31]:
# 11a. Add a marker layer for each city to the map.
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))